In [7]:
import os
import json
import glob
from openai import OpenAI
from tqdm import tqdm  # Widget hatası almamak için standart tqdm kullanıyoruz

# --- AYARLAR ---
# Buraya OpenAI API Key'inizi yapıştırın (sk-proj-... ile başlar)
OPENAI_API_KEY = "key_gelecek"

client = OpenAI(api_key=OPENAI_API_KEY)

# Klasör Yolları
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"

# Processed klasörü yoksa oluştur
os.makedirs(PROCESSED_DIR, exist_ok=True)

print("✅ OpenAI istemcisi hazır.")

✅ OpenAI istemcisi hazır.


In [9]:
def classify_with_openai(question_text, options, year):
    """
    Soruyu OpenAI API'ye gönderip analiz eder.
    """

    system_prompt = """
    Sen uzman bir Türkçe öğretmenisin. Verilen LGS (Liselere Geçiş Sınavı) Türkçe sorusunu analiz et.

    Çıktı Formatı (Kesinlikle sadece bu JSON yapısını döndür):
    {
        "ana_konu": "Konu Başlığı (Örn: Paragrafta Anlam, Sözcükte Anlam, Dil Bilgisi, Yazım Kuralları, Sözel Mantık, Grafik Yorumlama)",
        "alt_konu": "Detaylı Konu (Örn: Ana Düşünce, Yardımcı Düşünce, Fiilimsiler, Noktalama İşaretleri)",
        "kazanim": "Ölçülen Beceri (Örn: Çıkarım Yapma, Görsel Okuma, Metinler Arası İlişki Kurma)",
        "tahmini_zorluk": "1 (Çok Kolay) ile 5 (Çok Zor) arası sayı"
    }
    """

    user_content = f"""
    Lütfen şu soruyu analiz et:
    Yıl: {year}
    Soru Metni: {question_text}
    Seçenekler: {options}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", # En iyi fiyat/performans modeli. Daha yüksek zeka için "gpt-4o" yazabilirsiniz.
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ],
            response_format={ "type": "json_object" }, # JSON garantisi
            temperature=0.2 # Tutarlı sonuçlar için düşük sıcaklık
        )

        # Gelen metni JSON objesine çevir
        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        print(f"⚠️ Hata: {e}")
        return {
            "ana_konu": "Bilinmiyor",
            "alt_konu": "Hata",
            "kazanim": "Hata",
            "tahmini_zorluk": 0
        }

In [10]:
# Raw klasöründeki tüm dosyaları bul
json_files = sorted(glob.glob(os.path.join(RAW_DIR, "*.json")))

print(f"📂 Bulunan dosya sayısı: {len(json_files)}")

for file_path in json_files:
    filename = os.path.basename(file_path)
    year = filename.replace(".json", "") # Dosya adından yılı al

    print(f"\n🚀 {year} yılı işleniyor...")

    # Dosyayı oku
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_questions = []

    # O yılın sorularını dön
    # tqdm ile ilerlemeyi gösteriyoruz
    for question in tqdm(data.get("questions", []), desc=f"{year} Soruları"):

        q_text = question.get("text", "")
        q_options = question.get("options", [])

        # --- API'YE SOR ---
        analysis = classify_with_openai(q_text, q_options, year)

        # Veriyi birleştir
        new_record = question.copy()
        new_record["analysis"] = analysis

        processed_questions.append(new_record)

    # Dosya formatını hazırla
    processed_data = {
        "year": data.get("year"),
        "course": data.get("course"),
        "total_questions": len(processed_questions),
        "questions": processed_questions
    }

    # Processed klasörüne kaydet
    output_path = os.path.join(PROCESSED_DIR, filename)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=2)

    print(f"✅ {year} tamamlandı ve kaydedildi.")

print("\n🎉 Tüm yıllar başarıyla işlendi!")

📂 Bulunan dosya sayısı: 8

🚀 2018 yılı işleniyor...


2018 Soruları: 100%|██████████| 20/20 [00:34<00:00,  1.71s/it]


✅ 2018 tamamlandı ve kaydedildi.

🚀 2019 yılı işleniyor...


2019 Soruları: 100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


✅ 2019 tamamlandı ve kaydedildi.

🚀 2020 yılı işleniyor...


...

✅ 2023 tamamlandı ve kaydedildi.

🚀 2024 yılı işleniyor...


2024 Soruları: 100%|██████████| 20/20 [00:30<00:00,  1.53s/it]


✅ 2024 tamamlandı ve kaydedildi.

🚀 2025 yılı işleniyor...


2025 Soruları: 100%|██████████| 20/20 [00:30<00:00,  1.53s/it]

✅ 2025 tamamlandı ve kaydedildi.

🎉 Tüm yıllar başarıyla işlendi!


In [11]:
import pandas as pd

# Tüm işlenmiş dosyaları birleştirip tabloya dökelim
all_records = []
processed_files = glob.glob(os.path.join(PROCESSED_DIR, "*.json"))

for p_file in processed_files:
    with open(p_file, 'r', encoding='utf-8') as f:
        p_data = json.load(f)
        for q in p_data['questions']:
            # Eğer analiz hatası varsa 'Bilinmiyor' gelebilir, onları filtreleyebiliriz
            all_records.append({
                "Yıl": p_data['year'],
                "Soru_ID": q['id'],
                "Ana_Konu": q['analysis'].get('ana_konu'),
                "Alt_Konu": q['analysis'].get('alt_konu'),
                "Zorluk": q['analysis'].get('tahmini_zorluk')
            })

df = pd.DataFrame(all_records)

# İlk 5 satır
print(df.head())

# Hangi konudan kaç soru var?
print("\n--- Konu Dağılımı ---")
print(df['Ana_Konu'].value_counts())

    Yıl  Soru_ID          Ana_Konu          Alt_Konu  Zorluk
0  2022        1    Sözcükte Anlam  Anlam İlişkileri       3
1  2022        2  Paragrafta Anlam       Ana Düşünce       3
2  2022        3  Paragrafta Anlam       Ana Düşünce       3
3  2022        4  Paragrafta Anlam       Ana Düşünce       3
4  2022        5  Paragrafta Anlam       Ana Düşünce       3

--- Konu Dağılımı ---
Ana_Konu
Paragrafta Anlam    77
Sözel Mantık        27
Dil Bilgisi         17
Sözcükte Anlam      13
Yazım Kuralları     13
Grafik Yorumlama    10
Fiilimsiler          1
Cümle Türleri        1
Söz Sanatları        1
Name: count, dtype: int64
